In [ ]:
# apply Jupyter notebook style
from IPython.core.display import HTML

from custom.styles import style_string

HTML(style_string)

<div style="text-align:center;">
  <img src="custom/molssi_main_horizontal.png" style="display: block; margin: 0 auto; max-height:200px;">
</div>

Python Data Science Libraries - Pandas and Seaborn
===================================================

<div class="overview admonition"> 
<p class="admonition-title">Overview</p>

Questions:

* How can I use pandas to examine data?

* How can I use seaborn to visualize relationships?

Objectives:

* Learn the basics of pandas dataframes

* Use seaborn to make plots.

</div>


Pandas is a Python library used for data analysis and manipulation. Within the world of data science, it is a ubiquitous and widely used library. If you are learning how to analyze data in Python, it will be almost impossible to avoid pandas. 

Pandas is very powerful. In this session, we'll be learning how to access information in pandas dataframes and how to do some basic manipulation and analysis. The first step in using pandas is importing it. Typically when pandas is imported, it is shortened to `pd`.

In [ ]:
import pandas as pd

The first data set we are looking at in this workshop is solubility data. 
This is a data set which gives molecular descriptors along with solubility calculated using the ESOL method. 
The data was obtained from [moleculenet](https://moleculenet.org/datasets-1) and you can read more about the method [here](https://www.moreisdifferent.com/assets/DIY_drug_discovery/Delaney_paper.pdf). 
The dataset has been downloaded and placed in your `data` folder.

Later in the workshop, we'll be seing how we can fit this model ourselves using Python, but for now we are just going to examine the data using pandas. 
The data is saved in a csv file. CSV stands for "comma separated values". 
This is a file format where commas separate values. 
You can also typically read files like this into excel. 
In fact, initially in this session you may not yet see what advantages pandas offers over Excel. 
However, functionalites in pandas will be more useful once we start cleaning and fitting data.

In [ ]:
df = pd.read_csv("data/delaney-processed.csv")

## Examining the data

Now that we have used the pandas `read_csv` function, our data is in a variable called `df`. This variable is a **pandas dataframe**. This means it has lots of special functionalities thanks to pandas. For example, you can preview the first 5 rows using the `.head` function.

In [ ]:
df.head()

The `.info` function will give information about the columns and the data type of those columns. The data type will become very important later as we work with more data.

In [ ]:
df.info()

Pandas assigns data types to columns, and will do its best to decide the data column for each column based on what is in the column. 
You will see that this dataframe has `1128` values in each column. 

We see that the column `Minimum Degree` has the data type of `int64`. Here, `int` means `integer` and `64` means `64 bit`.  The `64 bit` refers to the amount of computer memory the variable can occupy. It won't really be important for us. Similarly, `float64` means `64 bit floating point`. These are decimal numbers.

The other column names which read `object` are not numeric. They might be strings or they might be something else. We'll discuss more later.

The `describe` function can be used on a dataframe to quickly see statistics about columns with numerical data. If you look at the columns that statistics are computed for and compare to the data type shown from `info`, you will see that we only get statistics for columns which had `int64` or `float64` data types.

In [ ]:
df.describe()

## Accessing Data

One great thing about pandas dataframes is how easy it is to access information. Pandas allows you to access information in a dataframe using both data indexes and names.

Pandas dataframes have rows and columns, you can see how many rows and columns using `.shape`. This will return the shape as `(num_rows, num_columns)`.

In [ ]:
df.shape

### Accessing with rows and column numbers 

#### The `.iloc` function
Access on a particular row and column using `.iloc` followed by square brackets and the row and column numbers you want to access. If you only put one number, it will be assumed to be the row number you want.

The following gets row number 35.

In [ ]:
df.iloc[35]

We can also specify a number of columns by adding a second number to the slicing.

In [ ]:
df.iloc[35, :3]

### Accessing with names

Each dataframe has an index (rows) and columns. The rows and columns have names. For the columns, these are indicated in `.head` by the bold row at the top of the dataframe. The row names are similarly listed in bold in the left of the dataframe. The index can be named, but by default it is usually just numbered. 

You can see information about the index or the columns using `df.index` or `df.columns`.

In [ ]:
df.index

In [ ]:
df.columns

Some of these column names are very long, and we might not want to type them 
when we want to reference data. 
We will rename some of these columns to make accessing the data more convenient.

In [ ]:
df.rename( columns = {
    "ESOL predicted log solubility in mols per litre": "ESOL solubility (mol/L)",
    "measured log solubility in mols per litre" : "measured solubility (mol/L)"
}, inplace=True)

To access part of a dataframe using the index or column names, would use the `loc` function.

Because our index names are just numbers, this doesn't look that different than `iloc` when we are accessing rows.

#### The `.loc` function

In [ ]:
df.loc[35]

However, we can now use column names

In [ ]:
df.loc[35, "smiles"]

#### Using column names

You can pull a column of several columns of information using the syntax

```python
df[column_name]
```

To pull several columns, do

```python
df[[colname1, colname2]]
```

For example

In [ ]:
df["Compound ID"]

In [ ]:
df[["Compound ID", "smiles"]]

## Filtering Data

There are two ways filtering in dataframes are typically done. Both are shown here for completeness, and because you will see both when looking at code others have written.

In the first method, the strategy is that you first establish where something is true within a dataframe, then use that to filter the dataframe.

In [ ]:
df["Polar Surface Area"] >17

This gives us a list of values with either `True` or `False` that can be used to index into the dataframe.

In [ ]:
df[df["Polar Surface Area"] > 17]

## Sorting Data

Pandas allows you to easily sort data using the `sort_values` method. Inside of the function call you list the column you would like to sort by. By default, the values will be sorted from lowest to highest (or `ascending`). This method will `return` a sorted dataframe.

In [ ]:
df.sort_values("Number of H-Bond Donors")

If you include more than one column name, the dataframe will be sorted by multiple columns. First, it will be sorted by the first column indicated, then that sort will be sorted by the second. Consider the following examples. We first sort by `Number of H-Bond Donors`. Within that sort, we sort by `Molecular Weight`. 

In [ ]:
df_sorted = df.sort_values(["Number of H-Bond Donors", "Molecular Weight"])
df_sorted.head()

Note the bolded indices to the left of the dataframe. After we perform our sort, these are no longer in order. In the sorted dataframe, `loc` and `iloc` do not return the same values using the same number.

In [ ]:
df_sorted.loc[934]

In [ ]:
df_sorted.iloc[934]

## Performing Operations on Multiple Cells

### Broadcasting

Pandas dataframes have the convenient feature that they use something called `broadcasting`. This means that if you are doing something like subtracting a number, multiplying, etc to a column or dataframe of information, it can be done all at once instead of with a `for` loop. Consider if we wanted to express the molecular surface area in $nm^2$ instead of Å$^2$. To do this, we would need to divide each value by 100.

Instead of writing a `for` loop that does this, we can just write the following code. This will return a pandas Series (one dimensional dataframe). 

In [ ]:
df["Polar Surface Area"] / 100

To save it as a new column, we need to capture the output in a column. You can create a new column in a dataframe using the following syntax.

In [ ]:
df["Polar Surface Area (nm^2)"] = df["Polar Surface Area"] / 100

You can also add, subtract, or multiply two columns with one another. For example, we might want to calculate the difference between the predicted and observed values for this solubility model.

In [ ]:
df["ESOL solubility (mol/L)"] - df["measured solubility (mol/L)"]


<div class="exercise admonition">
<p class="admonition-title">Check Your Understanding</p>
<p>  Save the difference between the predicted and measured solubility in a column named "difference". </p>
</div>



### The `.apply` method - for rows and columns

The apply method is used to apply a function to either the rows or columns of a dataframe.  If you use this on a single column, the function will be applied to every value.

In [ ]:
df["Compound ID"].apply(len)

In [ ]:
df[["smiles", "Compound ID"]].apply(len)


<div class="exercise admonition">
<p class="admonition-title">Check Your Understanding</p>
<p>Some libraries from RDKit are imported below. PandasTools for RDKit will be fully explained in the next lesson. Use the apply function to apply Chem.MolFromSmiles to the smiles column of your dataframe. This will create a list of RDKit molecule objects. Save this in a new column called "molecule"</p>
</div>


In [ ]:
from rdkit import Chem

from rdkit.Chem import PandasTools

PandasTools.RenderImagesInAllDataFrames(True)

In [ ]:
# Your code here


In [ ]:
df.head()

## Visualization with seaborn

If you have programmed with Python before, you are probably familiar with the library matplotlib.
In this workshop, we will use a library called seaborn. 
Seaborn is built on top of matplotlib and is commoly used for statistical visualizations.

Seaborn [regplot](https://seaborn.pydata.org/generated/seaborn.regplot.html) can be good for seeing linear relationships.
It will plot the data of interest with a linear regression.

In [ ]:
import seaborn as sns

sns.regplot(x="Molecular Weight", y="measured solubility (mol/L)", data=df)

We can also quickly get a glance at the relationship between variables using a pandas correlation matrix (`df.corr(numeric_only=True))`) and a seaborn heatmap.

In [ ]:
sns.heatmap(df.corr(numeric_only=True), cmap="Blues", annot=True)